<a href="https://colab.research.google.com/github/Kish-ie/RNN-s-sequential-modelling/blob/main/RNN_sequential_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#APOLLO KIIRU IN14/00006/21
#ALEX KAMAU IN13/00005/21
#MWANGI ANTHONY IN13/00035/21
#SIMON NJAU IN14/00101/21
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib
from tensorflow.keras.callbacks import EarlyStopping

# Sample data
data = {
    "user_id": ["U1", "U1", "U1", "U2", "U2", "U3", "U3", "U3", "U4", "U4"],
    "product_id": ["P1", "P3", "P4", "P1", "P2", "P3", "P4", "P5", "P2", "P3"],
}

df = pd.DataFrame(data)

# Encoding user and product IDs
user_encoder = LabelEncoder()
product_encoder = LabelEncoder()

df["user_id"] = user_encoder.fit_transform(df["user_id"])
df["product_id"] = product_encoder.fit_transform(df["product_id"])

# Grouping by user and creating sequences
user_groups = df.groupby("user_id")["product_id"].apply(list)
sequences = user_groups.tolist()

sequence_length = 2
X, y = [], []

for seq in sequences:
    for i in range(len(seq) - sequence_length):
        X.append(seq[i : i + sequence_length])
        y.append(seq[i + sequence_length])

# Padding sequences to ensure uniform length
X = pad_sequences(X, maxlen=sequence_length, padding='pre', truncating='pre')
y = np.array(y)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Number of unique products
num_products = len(product_encoder.classes_)

# Building a simpler model
model = Sequential([
    Embedding(input_dim=num_products, output_dim=16),
    GRU(16, return_sequences=False),
    Dropout(0.2),
    Dense(num_products, activation="softmax"),
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training the model
model.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Function to recommend the next product
def recommend_next_product(user_id):
    try:
        encoded_user = user_encoder.transform([user_id])[0]
        user_seq = user_groups[encoded_user][-sequence_length:]
        user_seq = pad_sequences([user_seq], maxlen=sequence_length, padding='pre', truncating='pre')
        predicted_product_id = np.argmax(model.predict(user_seq), axis=-1)[0]
        recommended_product = product_encoder.inverse_transform([predicted_product_id])[0]
        return recommended_product
    except KeyError:
        print(f"User ID {user_id} not found in the training data.")
        return None

# Testing the recommendation function
test_users = ["U1", "U2", "U3", "U4"]
for user in test_users:
    recommended_product = recommend_next_product(user)
    print(f"\nRecommended product for user {user}: {recommended_product}")

# Save the model and encoders
model.save('product_recommendation_model.keras')  # Using native Keras format
joblib.dump(user_encoder, 'user_encoder.pkl')
joblib.dump(product_encoder, 'product_encoder.pkl')

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.0000e+00 - loss: 1.6098 - val_accuracy: 0.0000e+00 - val_loss: 1.5952
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step - accuracy: 0.0000e+00 - loss: 1.6006 - val_accuracy: 0.0000e+00 - val_loss: 1.5975
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step - accuracy: 1.0000 - loss: 1.6007 - val_accuracy: 1.0000 - val_loss: 1.5996
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 1.0000 - loss: 1.5847 - val_accuracy: 0.0000e+00 - val_loss: 1.6016
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 1.0000 - loss: 1.5730 - val_accuracy: 0.0000e+00 - val_loss: 1.6036
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - accuracy: 1.0000 - loss: 1.5675 - val_accuracy: 0.0000e+00 - val_loss: 1.6057
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 744ms/step

Recommended product for user U1: P1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

Recommended product for user U2: P4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

Recommended product for user 

['product_encoder.pkl']